# Possible New Geometry - Cropped Image

With geom_crop(), you can fill polygons with any raster images loaded from a URL.

To run the notebook, you need to log in on [pixabay.com](https://pixabay.com) and get the API key from [here](https://pixabay.com/api/docs).

In [ ]:
import requests

import pandas as pd
import geopandas as gpd
import shapely

from lets_plot import *
LetsPlot.setup_html()
from geoms.crop import geom_crop

## Preparation

In [ ]:
pixabay_api_key = input('Type your API key: ')

In [ ]:
def project_geometry(gdf, geometry, *, proj, new_width, new_height):
    gdf = gdf.to_crs(proj)
    origin = (gdf.total_bounds[0], gdf.total_bounds[1])
    gdf[geometry] = gdf[geometry].apply(lambda geom: shapely.affinity.translate(geom, \
                                                                                xoff=-origin[0], \
                                                                                yoff=-origin[1]))
    current_width = gdf.total_bounds[2] - gdf.total_bounds[0]
    current_height = gdf.total_bounds[3] - gdf.total_bounds[1]
    gdf[geometry] = gdf[geometry].apply(lambda geom: shapely.affinity.scale(geom, \
                                                                            xfact=new_width / current_width, \
                                                                            yfact=new_height / current_height, \
                                                                            origin=(0, 0)))
    gdf[geometry] = gdf[geometry].apply(lambda geom: shapely.wkt.loads(shapely.wkt.dumps(geom, rounding_precision=0)))
    return gdf

In [ ]:
def get_europe_gdf():
    world_gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    europe_bb = gpd.GeoDataFrame(geometry=[ shapely.geometry.box(-10, 70, 40, 30) ], crs=world_gdf.crs)
    return gpd.overlay(world_gdf, europe_bb, how='intersection')

In [ ]:
def get_projected_europe_gdf(*, width, height):
    europe_gdf = get_europe_gdf()
    europe_gdf = project_geometry(europe_gdf, 'geometry', proj='epsg:3857', new_width=width, new_height=height)
    europe_gdf = europe_gdf[~europe_gdf.geometry.isna()]
    europe_gdf.geometry = europe_gdf.geometry.apply(lambda geom: geom if geom.is_valid else geom.convex_hull)
    return europe_gdf

In [ ]:
def point_to_df(geom, *, name):
    return pd.DataFrame(dict(x=[geom.x], y=[geom.y], name=[name], index=[0]))

def polygon_to_df(geom, *, name, index=0):
    x, y = list(zip(*geom.exterior.coords))
    return pd.DataFrame(dict(x=x, y=y, name=[name] * len(x), index=[index] * len(x)))

def multipolygon_to_df(geom, *, name):
    return pd.concat([polygon_to_df(polygon, name=name, index=index) for index, polygon in enumerate(geom)])

def geom_to_df(geom, *, name):
    if geom.type == 'Point':
        return point_to_df(geom, name=name)
    elif geom.type == 'Polygon':
        return polygon_to_df(geom, name=name)
    elif geom.type == 'MultiPolygon':
        return multipolygon_to_df(geom, name=name)
    else:
        return pd.DataFrame(dict(x=[], y=[], name=[], index=[]))

def gdf_to_df(gdf):
    df = pd.concat([geom_to_df(row['geometry'], name=row['name']) for i, row in gdf.iterrows()])
    return df.reset_index(drop=True)

In [ ]:
def get_image_url(country, *, category):
    query = country
    request_url = 'https://pixabay.com/api/?key={api_key}&category={category}&q={query}'.format(
        api_key=pixabay_api_key,
        category=category,
        query=query.replace(' ', '+')
    )
    hits = requests.get(request_url).json().get('hits')
    if hits and len(hits) > 0:
        return hits[0]['largeImageURL']
    else:
        return ''

In [ ]:
def construct_images_df(countries_df, *, category, column_name=None):
    images_df = countries_df.copy()
    column_name = column_name or '%s_img' % category
    images_df[column_name] = countries_df.iloc[:, 0].apply(get_image_url, category=category)
    return images_df

## Let's Plot!

In [ ]:
width, height = 450, 600

In [ ]:
europe_gdf = get_projected_europe_gdf(width=width, height=height)
europe_gdf = europe_gdf.merge(construct_images_df(europe_gdf[['name']], category='nature'), on='name')
europe_gdf = europe_gdf.merge(construct_images_df(europe_gdf[['name']], category='buildings'), on='name')

In [ ]:
ggplot() + \
    geom_crop(aes(geometry='geometry', url='nature_img'), europe_gdf) + \
    geom_polygon(aes(x='x', y='y', group='name'), data=gdf_to_df(europe_gdf), \
                 color='black', size=1, alpha=0, tooltips=layer_tooltips().line('@name')) + \
    ggtitle('Nature in Europe') + \
    ggsize(width, height) + \
    theme(legend_position='none', axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank')

In [ ]:
ggplot() + \
    geom_crop(aes(geometry='geometry', url='buildings_img'), europe_gdf) + \
    geom_polygon(aes(x='x', y='y', group='name'), data=gdf_to_df(europe_gdf), \
                 color='black', size=1, alpha=0, tooltips=layer_tooltips().line('@name')) + \
    ggtitle('Buildings in Europe') + \
    ggsize(width, height) + \
    theme(legend_position='none', axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank')